# Goal 

implement sequence assembly algorithm to assemble fragmented reads 

## Resources 
Github code: https://github.com/BenLangmead/ads1-notebooks/blob/master/4.02_GreedySCS.ipynb

merging strings: https://www.geeksforgeeks.org/python-merging-two-strings-with-suffix-and-prefix/?ref=rp


In [1]:
import Bio
from Bio import SeqIO
from Bio.Seq import Seq

import numpy as np
import pandas as pd
import csv

In [2]:
df = pd.read_csv('fragmented_reads_table.csv')

In [3]:
df['Count'] = len(df['Sequence'])
df

,Unnamed: 0,ID,Sequence,Count
0,0,1,CCCTGTCTACCACCCAGACTATCGTGTAGTTCTGCCTGTTCCGTAA...,127
1,1,2,AGACTATCGTGTAGTTCTGCCTGTTCCGTAAGTCGTAGATTGCTAT...,127
2,2,3,TCTGCCTGTTCCGTAAGTCGTAGATTGCTATCCTGGAAATCATCGT...,127
3,3,4,AGTCGTAGATTGCTATCCTGGAAATCATCGTGCTCAGGATGTTAAT...,127
4,4,5,TCCTGGAAATCATCGTGCTCAGGATGTTAATATCTAGCGTCCTACG...,127
...,...,...,...,...
122,122,123,ATTTATATGACGGCCAAAAATGGAGATATTATAGTCGACCAAGTAT...,127
123,123,124,AAAAATGGAGATATTATAGTCGACCAAGTATTGGCGTCGAACAACC...,127
124,124,125,ATAGTCGACCAAGTATTGGCGTCGAACAACCGCGCCCTGCAGAATC...,127
125,125,126,TTGGCGTCGAACAACCGCGCCCTGCAGAATCCCAAGATTCGCCAGG...,127


In [4]:
seq_list = df['Sequence'].to_list()

In [5]:
seq_list

['CCCTGTCTACCACCCAGACTATCGTGTAGTTCTGCCTGTTCCGTAAGTCGTAGATTGCTATCCTGGAAATCATCGTGCTCAGGATGTTAATATCTAGCGT',
 'AGACTATCGTGTAGTTCTGCCTGTTCCGTAAGTCGTAGATTGCTATCCTGGAAATCATCGTGCTCAGGATGTTAATATCTAGCGTCCTACGTTACGAGTT',
 'TCTGCCTGTTCCGTAAGTCGTAGATTGCTATCCTGGAAATCATCGTGCTCAGGATGTTAATATCTAGCGTCCTACGTTACGAGTTGGCAGATGACAGATC',
 'AGTCGTAGATTGCTATCCTGGAAATCATCGTGCTCAGGATGTTAATATCTAGCGTCCTACGTTACGAGTTGGCAGATGACAGATCGTAGTCGTGGTAAGG',
 'TCCTGGAAATCATCGTGCTCAGGATGTTAATATCTAGCGTCCTACGTTACGAGTTGGCAGATGACAGATCGTAGTCGTGGTAAGGGGCATTGCCGCTTGT',
 'TGCTCAGGATGTTAATATCTAGCGTCCTACGTTACGAGTTGGCAGATGACAGATCGTAGTCGTGGTAAGGGGCATTGCCGCTTGTGACCCAGTTCGCGTG',
 'TATCTAGCGTCCTACGTTACGAGTTGGCAGATGACAGATCGTAGTCGTGGTAAGGGGCATTGCCGCTTGTGACCCAGTTCGCGTGCCTAGCAGCACTCCA',
 'GTTACGAGTTGGCAGATGACAGATCGTAGTCGTGGTAAGGGGCATTGCCGCTTGTGACCCAGTTCGCGTGCCTAGCAGCACTCCAAAATAAAGTTTACAG',
 'ATGACAGATCGTAGTCGTGGTAAGGGGCATTGCCGCTTGTGACCCAGTTCGCGTGCCTAGCAGCACTCCAAAATAAAGTTTACAGTACCGTCCGGACGGC',
 'CGTGGTAAGGGGCATTGCCGCTTGTGACCCAGTTCGCGTGCCTAGCAGCACTC

In [6]:
dummy_list = ['ABCDEFGH', 'CDEFGHI', 'CDEFGHI','HIJKLMNOP']

In [7]:
dummy_list[0][:3]

'ABC'

# Overlap 

find overlapping reads ... where prefix matches suffix 

In [8]:
# The overlap function needs to take in a sequence A and a sequence B. 
# We want the maximum overlap so we must loop through all cases 
# We want it to return the length of the overlap? 

def overlap2 (a, b, match = 3): 
    #want to start at position 0 
    pos = 0
    
    #search until end
    while True:
        
        # find a match for a to b 
        # by looking at the next three characters in b
        # by default, this will end at the end of the string a
        # pos gives where the match occurs in the string 
        pos = a.find(b[:match], pos)
     
        if b.startswith(a[pos:]):
            return(len(a[pos:]))
        if(pos == -1):
            return 0
        pos += 1

In [9]:
overlap2('ABCDE', 'CDEFGHIJ')

3

In [10]:
string1 = 'ABCDE'
string2 = 'CDEFGHIJ'
string1[:(5-3)] + string2

'ABCDEFGHIJ'

In [11]:
from itertools import permutations

def shortest_superstring (slist):
    shortest_string = None
    overlap = 0
    # find the largest overlap 
    for combo in itertools.permutations(slist):
        print(combo)
        sup = combo[0]
        for i in range(len(slist)-1): 
            overlap_temp = overlap2(combo[i], combo[i+1])
            if (overlap_temp > overlap): 
                overlap = overlap_temp 
                string1 = combo[0]
                string2 = combo[1]
            sup += string1[:(len(string1) - overlap)] + string2
        if shortest_string is None or (len(sup) < len(shortest_string)): 
            shorest_string = sup
    return(shorest_string)

shortest_superstring(['ABCDE', 'CDEFGHIJ', 'JKLMO', 'CDEFGHIJ'])

# Layout 

take overlaps and bundle them to contigs 

# Consenus 

check and correct errors

In [12]:
def overlap(a, b, min_length=3):
    """ Return length of longest suffix of 'a' matching
        a prefix of 'b' that is at least 'min_length'
        characters long.  If no such overlap exists,
        return 0. """
    start = 0  # start all the way at the left
    while True:
        start = a.find(b[:min_length], start)  # look for b's prefix in a
        if start == -1:  # no more occurrences to right
            return 0
        # found occurrence; check for full suffix/prefix match
        if b.startswith(a[start:]):
            return len(a)-start
        start += 1  # move just past previous match

In [13]:
import itertools

def scs(ss):
    """ Returns shortest common superstring of given strings,
        assuming no string is a strict substring of another """
    shortest_sup = None
    for ssperm in itertools.permutations(ss):
        sup = ssperm[0]  # superstring starts as first string
        for i in range(len(ss)-1):
            # overlap adjacent strings A and B in the permutation
            olen = overlap(ssperm[i], ssperm[i+1], min_length=1)
            # add non-overlapping portion of B to superstring
            #sup += ssperm[i+1][-(len(ssperm[i+1])-olen):]
            sup += ssperm[i+1][olen:]
        if shortest_sup is None or len(sup) < len(shortest_sup):
            shortest_sup = sup  # found shorter superstring
    return shortest_sup  # return shortest

In [14]:
#scs(seq_list)

In [15]:
scs(['ABCDE', 'CDEFGHIJ', 'JKLMO', 'CDEFGHIJ'])

'ABCDEFGHIJKLMO'

In [16]:
def pick_maximal_overlap(reads, k):
    """ Return a pair of reads from the list with a
        maximal suffix/prefix overlap >= k.  Returns
        overlap length 0 if there are no such overlaps."""
    reada, readb = None, None
    best_olen = 0
    for a, b in itertools.permutations(reads, 2):
        olen = overlap(a, b, min_length=k)
        if olen > best_olen:
            reada, readb = a, b
            best_olen = olen
    return reada, readb, best_olen

In [17]:
def greedy_scs(reads, k):
    """ Greedy shortest-common-superstring merge.
        Repeat until no edges (overlaps of length >= k)
        remain. """
    read_a, read_b, olen = pick_maximal_overlap(reads, k)
    while olen > 0:
        reads.remove(read_a)
        reads.remove(read_b)
        reads.append(read_a + read_b[olen:])
        read_a, read_b, olen = pick_maximal_overlap(reads, k)
    return ''.join(reads)

In [18]:
greedy_scs(seq_list, 50)

'CCCTGTCTACCACCCAGACTATCGTGTAGTTCTGCCTGTTCCGTAAGTCGTAGATTGCTATCCTGGAAATCATCGTGCTCAGGATGTTAATATCTAGCGTCCTACGTTACGAGTTGGCAGATGACAGATCGTAGTCGTGGTAAGGGGCATTGCCGCTTGTGACCCAGTTCGCGTGCCTAGCAGCACTCCAAAATAAAGTTTACAGTACCGTCCGGACGGCAGAACTGTCCTCTAGATCGTCCTAACGCCTTAGTCGAATCCCTTGCCGTCGGTAACCACTGAATAAACTACGCGTTAGGACTTTGTCAGACGCGAGGAGCTAGTAGGAGGACAAATCAGCAAACGACCCTGAATTGAACAATGTGAGTAGGTATAACTGTGCTTGTATGACGTCCCGTTCGGTCGTTCTTGAGCAACTTCGGCCAGTGCATGCTATGGGGGAAGCTATGAATTCTATGTTGGAACTTGGGCCCGGCATAGTAGTTTATGCCTGTGGACCGGTGTTGAGTGTATCTGCTGGACCCCGGCGCGTTCACCTGTCCACATCTAATCCAAACATATACTATTGGTATTTGAGCGTCTCACAACGACATCGACTGGTATTAGACACCTACCAGGAACAACCAATCGGTTTAGATGACGCACAGCCACGGACAGCCTCTGTTGCTTGAGCAGTCCCAAAGTGCGTACCTGAAGCCTGCCAAAACGTAGCCTAGGCAAATGCCCGTCGTCTTGCTCATAACTCCTTGGGACTGGCGTATCCATAAATAATCCATTCGATTCCTTGAGAGTTCCACATTAGAGACTTATCCATCGAGGATCAGGCCAAATCCGCGAGACCCGACCGAGATCAAGTATAACTCATTACGCGTGGTGTGGTTGCGGCCCACCCTTATCGTGAGCCAGTTGTTGGATATACCCCTGGGCGGGCCTAAAGCTCCGCAACGAACACCCCCTCCGCTGTGTCTGGTCGATTCTGGCTAGCCGCTCCGTTTGGGT

In [19]:
len(greedy_scs(seq_list, 50))

1990